# Required instalations and imports

In [18]:
!pip install selenium

In [19]:
!pip install beautifulsoup4

In [20]:
!pip install requests

In [1]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import datetime as dt
import time
import decimal
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from IPython.core.display import HTML
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

# BATTTLEFLY

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows Phone 10.0; Android 6.0.1; Microsoft; RM-1152)'}

## We access to the api that delivers us the information about tournament schedule

In [4]:
def get_urls_tournaments(start_date,end_date,url):
    tournaments = []
    while dt.datetime.strptime(start_date,date_format) < dt.datetime.strptime(end_date,date_format):    
        response = requests.get(url).json()
        for tournament in response['tournaments']:           
            if 'Masters Tour Madrid' in tournament['name']:
                tinfo = {'id':tournament['_id'],'name':tournament['name']}
                print(tinfo)
                tournaments.append(tinfo)
        if not 'start' in response['next']:
            return tournaments
        start_date = response['next']['start'][:-5]
        url = f'https://search.battlefy.com/tournament/browse/hearthstone?end={end_date}&start={start_date}'
        time.sleep(2.4)
        print(url)

In [5]:
start_date = '2020-08-06T00:00:00'
date_format = '%Y-%m-%d'+'T'+'%H:%M:%S'
end_date = dt.datetime.now().strftime(date_format)
url = f'https://search.battlefy.com/tournament/browse/hearthstone?end={end_date}&start={start_date}'
print(url)
tnames = []
tnames = get_urls_tournaments(start_date,end_date,url)
print(tnames)



https://search.battlefy.com/tournament/browse/hearthstone?end=2020-08-21T12:02:53&start=2020-08-06T00:00:00
{'id': '5f0d6a35bd784e7392dd336a', 'name': 'Hearthstone Masters Qualifier - Masters Tour Madrid - #1'}
{'id': '5f12435e0770843f77f7c3be', 'name': 'Hearthstone Masters Qualifier - Masters Tour Madrid - #2'}
{'id': '5f12435e0770843f77f7c3bf', 'name': 'Hearthstone Masters Qualifier - Masters Tour Madrid - #3'}
{'id': '5f12435e0770843f77f7c3c0', 'name': 'Hearthstone Masters Qualifier - Masters Tour Madrid - #4'}
{'id': '5f12435e0770843f77f7c3c1', 'name': 'Hearthstone Masters Qualifier - Masters Tour Madrid - #5'}
{'id': '5f12435e0770843f77f7c3c2', 'name': 'Hearthstone Masters Qualifier - Masters Tour Madrid - #6'}
{'id': '5f12435e0770843f77f7c3c3', 'name': 'Hearthstone Masters Qualifier - Masters Tour Madrid - #7'}
https://search.battlefy.com/tournament/browse/hearthstone?end=2020-08-21T12:02:53&start=2020-08-08T12:00:44
{'id': '5f12435e0770843f77f7c3c4', 'name': 'Hearthstone Masters

## From every tournament stracted we form the url where the tournament is actually stored, we want to access it because there we can obtain the stage, which is required in order to retrieve the results via api

In [7]:
url_tournament = 'https://battlefy.com/hsesports/'
tournament_urls = [] 
for tournament in tnames:
    tname = tournament['name'].replace(' - ','-').replace('#','').replace(' ','-').lower().strip()
    tournament_urls.append(f'{url_tournament}{tname}/{tournament["id"]}/info?infoTab=details')
print(tournament_urls)

['https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-1/5f0d6a35bd784e7392dd336a/info?infoTab=details', 'https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-2/5f12435e0770843f77f7c3be/info?infoTab=details', 'https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-3/5f12435e0770843f77f7c3bf/info?infoTab=details', 'https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-4/5f12435e0770843f77f7c3c0/info?infoTab=details', 'https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-5/5f12435e0770843f77f7c3c1/info?infoTab=details', 'https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-6/5f12435e0770843f77f7c3c2/info?infoTab=details', 'https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-7/5f12435e0770843f77f7c3c3/info?infoTab=details', 'https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-

## Declare all possible classes someone can play at HS

In [8]:
hs_classes = ['Paladin','Rogue','Mage','Druid','Demonhunter','Hunter','Priest','Shaman','Warlock','Warrior']

## For every tournament url, access it, then click on the stats tab, and then extract from the url the stage number and store it in a list

In [9]:
class Tournament:
    def __init__(self,name,date,matches):
        self.date = date
        self.name = name
        self.matchups = {}
        self.stats = []
        self.stats.append(['']+hs_classes+['Total'])
        for hs_class in hs_classes:
            self.matchups[hs_class] = {}
            for class_lvl2 in hs_classes:
                self.matchups[hs_class][class_lvl2] = {'win' : 0,'lost':0,'games_played':0}
        for result in matches:       
            self.matchups[result['stats']['top']['class'].capitalize()][result['stats']['bottom']['class'].capitalize()]['games_played']+=1         
            self.matchups[result['stats']['bottom']['class'].capitalize()][result['stats']['top']['class'].capitalize()]['games_played']+=1         
            if result['stats']['top']['winner']:
                self.matchups[result['stats']['top']['class'].capitalize()][result['stats']['bottom']['class'].capitalize()]['win']+=1
                self.matchups[result['stats']['bottom']['class'].capitalize()][result['stats']['top']['class'].capitalize()]['lost']+=1
            else:
                self.matchups[result['stats']['top']['class'].capitalize()][result['stats']['bottom']['class'].capitalize()]['lost']+=1
                self.matchups[result['stats']['bottom']['class'].capitalize()][result['stats']['top']['class'].capitalize()]['win']+=1
        for hs_class in hs_classes:
            new_stats_row = [hs_class]
            for class_lvl2 in hs_classes:
                if self.matchups[hs_class][class_lvl2]['games_played']:
                    new_stats_row.append(str(round(100*self.matchups[hs_class][class_lvl2]['win']/self.matchups[hs_class][class_lvl2]['games_played'],2))+f'%({self.matchups[hs_class][class_lvl2]["games_played"]})')
                else :
                    new_stats_row.append('-')
            class_total_games_played = sum([self.matchups[hs_class][opponent]['games_played'] for opponent in hs_classes])
            class_total_wins = sum([self.matchups[hs_class][opponent]['win'] for opponent in hs_classes])
            new_stats_row.append(str(round(100*class_total_wins/class_total_games_played,2))+f'%({class_total_games_played})')
            self.stats.append(new_stats_row)
            
    def  __str__(self):
        return f'Tournament Name : {self.name}\nDate:{self.date}\nMatchups:{self.matchups}\n\nStats:{self.stats}'
    
class TournamentGroup:
    def __init__(self,tournaments):
        self.tournaments = tournaments
        self.start_date = min([datetime.strptime(tournament.date, '%Y-%m-%d')  for tournament in tournaments])
        self.end_date = max([datetime.strptime(tournament.date, '%Y-%m-%d')  for tournament in tournaments])
        
    

In [15]:
stats_urls = []
#We set the options of starting chrome as maximized because this webpage doesn't show the tab "stats" so we have to maximize
# Chrome in order to be able to access it
options = Options() 
options.add_argument("--start-maximized") 
driver=webdriver.Chrome(chrome_options=options)
for url in tournament_urls:
    driver.get(url)
    #even tho we set the browser as maximized to access the component we need to wait for it to be clickable
    element = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="tournament"]/div/bf-tournament/div[2]/div/div[3]/div/div/div[3]/div/ul/li[6]/a[1]')))
    element.click();
    stats = driver.find_elements_by_xpath('//*[@id="tournament"]/div/bf-tournament/div[2]/div/div[3]/div/div/div[3]/div/ul/li[6]/a[1]')
    print(stats)
    stats[0].click()
    #We set a 2s sleep because it takes some time for the current url to be updated
    time.sleep(2)
    current_url = driver.current_url
    print(current_url)
    stage = re.search('stage\/(.*)\/',current_url).group()[6:]
    stage = stage[:-1]
    print(stage)
    stats_urls.append(f'https://dtmwra1jsgyb0.cloudfront.net/stages/{stage}/stats')


C:\Users\Alex\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


[<selenium.webdriver.remote.webelement.WebElement (session="48d4d6d6ac2ec92ecd871c40183a97de", element="b861aedd-cb52-44d0-9bb4-4b33560b166e")>]
https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-1/5f0d6a35bd784e7392dd336a/stage/5f12340feea71b24bc30f497/stats
5f12340feea71b24bc30f497
[<selenium.webdriver.remote.webelement.WebElement (session="48d4d6d6ac2ec92ecd871c40183a97de", element="e071180c-b25e-4797-a014-a72fd708ba38")>]
https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-2/5f12435e0770843f77f7c3be/stage/5f12435e0770843f77f7c417/stats
5f12435e0770843f77f7c417
[<selenium.webdriver.remote.webelement.WebElement (session="48d4d6d6ac2ec92ecd871c40183a97de", element="f0252071-60dc-4ca8-8931-7d4f746adcb7")>]
https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-3/5f12435e0770843f77f7c3bf/stage/5f12435e0770843f77f7c418/stats
5f12435e0770843f77f7c418
[<selenium.webdriver.remote.webelement.WebElement (sess

[<selenium.webdriver.remote.webelement.WebElement (session="48d4d6d6ac2ec92ecd871c40183a97de", element="93b630e1-1e54-469b-83ed-f472337fa8b5")>]
https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-27/5f12435e0770843f77f7c3d7/stage/5f12435e0770843f77f7c430/stats
5f12435e0770843f77f7c430
[<selenium.webdriver.remote.webelement.WebElement (session="48d4d6d6ac2ec92ecd871c40183a97de", element="1c3397b5-15cf-4675-b718-37d0ca1994da")>]
https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-28/5f12435e0770843f77f7c3d8/stage/5f12435e0770843f77f7c431/stats
5f12435e0770843f77f7c431
[<selenium.webdriver.remote.webelement.WebElement (session="48d4d6d6ac2ec92ecd871c40183a97de", element="6986fc5d-927d-4ad7-a637-8371361bb944")>]
https://battlefy.com/hsesports/hearthstone-masters-qualifier-masters-tour-madrid-29/5f12435e0770843f77f7c3d9/stage/5f12435e0770843f77f7c432/stats
5f12435e0770843f77f7c432
[<selenium.webdriver.remote.webelement.WebElement (s

In [11]:
print(stats_urls)

['https://dtmwra1jsgyb0.cloudfront.net/stages/5f12340feea71b24bc30f497/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c417/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c418/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c419/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c41a/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c41b/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c41c/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c41d/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c41e/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c41f/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c420/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77f7c421/stats', 'https://dtmwra1jsgyb0.cloudfront.net/stages/5f12435e0770843f77

## Extract data from all the urls and create a tournament record for every one of them

In [12]:
tournaments = []
i = 0
for url in stats_urls:
    tournament = []
    response_stats = requests.get(url).json()
    tournaments.append(Tournament(tnames[i],response_stats[0]['createdAt'][:10],response_stats))
    i+=1
results = {}

In [13]:
print(tournaments[-1])

Tournament Name : {'id': '5f12435e0770843f77f7c3dc', 'name': 'Hearthstone Masters Qualifier - Masters Tour Madrid - #32'}
Date:2020-08-21
Matchups:{'Paladin': {'Paladin': {'win': 4, 'lost': 4, 'games_played': 8}, 'Rogue': {'win': 8, 'lost': 15, 'games_played': 23}, 'Mage': {'win': 3, 'lost': 4, 'games_played': 7}, 'Druid': {'win': 11, 'lost': 5, 'games_played': 16}, 'Demonhunter': {'win': 10, 'lost': 9, 'games_played': 19}, 'Hunter': {'win': 8, 'lost': 5, 'games_played': 13}, 'Priest': {'win': 7, 'lost': 5, 'games_played': 12}, 'Shaman': {'win': 2, 'lost': 0, 'games_played': 2}, 'Warlock': {'win': 15, 'lost': 8, 'games_played': 23}, 'Warrior': {'win': 10, 'lost': 12, 'games_played': 22}}, 'Rogue': {'Paladin': {'win': 15, 'lost': 8, 'games_played': 23}, 'Rogue': {'win': 74, 'lost': 74, 'games_played': 148}, 'Mage': {'win': 23, 'lost': 20, 'games_played': 43}, 'Druid': {'win': 43, 'lost': 53, 'games_played': 96}, 'Demonhunter': {'win': 35, 'lost': 58, 'games_played': 93}, 'Hunter': {'win

In [14]:
for tournament in  tournaments:
    df = pd.DataFrame(tournament.stats)
    print()
    stripped_name = tournament.name['name'].strip()
    df.to_csv(f'output/{stripped_name}.csv',index=False)
    display(HTML(df.to_html()))

,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,-,42.86%(14),66.67%(3),62.5%(16),57.89%(19),57.14%(7),100.0%(4),66.67%(3),85.71%(7),40.0%(20),56.99%(93)
2,Rogue,57.14%(14),50.0%(36),55.56%(9),51.79%(56),67.44%(43),90.0%(10),42.86%(14),74.07%(27),48.28%(29),29.03%(31),54.65%(269)
3,Mage,33.33%(3),44.44%(9),50.0%(2),64.71%(17),27.27%(11),50.0%(2),50.0%(2),40.0%(5),28.57%(7),30.77%(13),42.25%(71)
4,Druid,37.5%(16),48.21%(56),35.29%(17),50.0%(48),56.36%(55),50.0%(22),50.0%(18),33.33%(9),63.64%(22),58.82%(34),50.84%(297)
5,Demonhunter,42.11%(19),32.56%(43),72.73%(11),43.64%(55),50.0%(38),60.0%(5),38.1%(21),81.82%(11),60.0%(25),45.45%(11),47.28%(239)
6,Hunter,42.86%(7),10.0%(10),50.0%(2),50.0%(22),40.0%(5),50.0%(2),62.5%(8),50.0%(4),100.0%(3),57.14%(7),47.14%(70)
7,Priest,0.0%(4),57.14%(14),50.0%(2),50.0%(18),61.9%(21),37.5%(8),50.0%(4),66.67%(3),60.0%(10),52.38%(21),52.38%(105)
8,Shaman,33.33%(3),25.93%(27),60.0%(5),66.67%(9),18.18%(11),50.0%(4),33.33%(3),50.0%(4),25.0%(12),33.33%(15),34.41%(93)
9,Warlock,14.29%(7),51.72%(29),71.43%(7),36.36%(22),40.0%(25),0.0%(3),40.0%(10),75.0%(12),50.0%(12),50.0%(20),46.26%(147)


,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(10),33.33%(15),27.27%(11),53.85%(26),33.33%(12),50.0%(10),71.43%(14),60.0%(5),27.27%(11),54.17%(24),47.1%(138)
2,Rogue,66.67%(15),50.0%(32),50.0%(12),48.72%(78),65.85%(41),62.5%(24),63.64%(44),66.67%(12),57.89%(38),41.18%(51),55.04%(347)
3,Mage,72.73%(11),50.0%(12),50.0%(6),56.76%(37),47.37%(19),23.08%(13),52.63%(19),40.0%(10),60.0%(10),31.82%(22),48.43%(159)
4,Druid,46.15%(26),51.28%(78),43.24%(37),50.0%(56),44.3%(79),56.25%(32),62.22%(45),55.0%(40),65.91%(44),66.22%(74),54.21%(511)
5,Demonhunter,66.67%(12),34.15%(41),52.63%(19),55.7%(79),50.0%(28),66.67%(12),34.38%(32),53.33%(15),42.86%(21),18.18%(33),45.21%(292)
6,Hunter,50.0%(10),37.5%(24),76.92%(13),43.75%(32),33.33%(12),50.0%(8),56.0%(25),40.0%(5),25.0%(16),40.0%(25),44.71%(170)
7,Priest,28.57%(14),36.36%(44),47.37%(19),37.78%(45),65.62%(32),44.0%(25),50.0%(66),55.56%(9),48.89%(45),67.62%(105),51.73%(404)
8,Shaman,40.0%(5),33.33%(12),60.0%(10),45.0%(40),46.67%(15),60.0%(5),44.44%(9),50.0%(8),26.32%(19),26.09%(23),40.41%(146)
9,Warlock,72.73%(11),42.11%(38),40.0%(10),34.09%(44),57.14%(21),75.0%(16),51.11%(45),73.68%(19),50.0%(18),44.0%(50),49.63%(272)


,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(8),52.38%(21),50.0%(12),48.39%(31),50.0%(18),80.0%(10),37.5%(16),50.0%(6),50.0%(14),45.45%(33),49.7%(169)
2,Rogue,47.62%(21),50.0%(50),35.71%(14),35.37%(82),50.0%(34),64.71%(17),40.91%(44),71.43%(14),44.44%(18),39.02%(41),44.48%(335)
3,Mage,50.0%(12),64.29%(14),50.0%(4),33.33%(30),53.33%(15),50.0%(4),16.67%(12),20.0%(5),36.36%(11),32.14%(28),39.26%(135)
4,Druid,51.61%(31),64.63%(82),66.67%(30),50.0%(64),56.52%(69),66.67%(30),41.03%(39),68.57%(35),55.0%(20),69.44%(72),59.53%(472)
5,Demonhunter,50.0%(18),50.0%(34),46.67%(15),43.48%(69),50.0%(32),57.14%(14),37.93%(29),46.15%(13),65.22%(23),34.78%(23),47.04%(270)
6,Hunter,20.0%(10),35.29%(17),50.0%(4),33.33%(30),42.86%(14),50.0%(2),54.55%(11),50.0%(2),37.5%(8),19.05%(21),34.45%(119)
7,Priest,62.5%(16),59.09%(44),83.33%(12),58.97%(39),62.07%(29),45.45%(11),50.0%(48),41.18%(17),53.85%(13),66.2%(71),59.0%(300)
8,Shaman,50.0%(6),28.57%(14),80.0%(5),31.43%(35),53.85%(13),50.0%(2),58.82%(17),50.0%(4),75.0%(8),27.27%(11),44.35%(115)
9,Warlock,50.0%(14),55.56%(18),63.64%(11),45.0%(20),34.78%(23),62.5%(8),46.15%(13),25.0%(8),50.0%(12),62.5%(24),49.67%(151)


,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(10),41.67%(12),37.5%(8),44.12%(34),36.36%(11),33.33%(9),71.43%(7),-,60.0%(5),56.0%(25),47.11%(121)
2,Rogue,58.33%(12),50.0%(16),77.78%(9),42.55%(47),58.33%(24),60.0%(5),56.52%(23),40.0%(5),42.86%(7),40.0%(25),50.29%(173)
3,Mage,62.5%(8),22.22%(9),50.0%(2),57.69%(26),46.67%(15),0.0%(1),44.44%(9),100.0%(1),0.0%(1),38.1%(21),46.24%(93)
4,Druid,55.88%(34),57.45%(47),42.31%(26),50.0%(66),51.06%(47),46.43%(28),40.48%(42),55.56%(18),60.0%(20),74.58%(59),54.26%(387)
5,Demonhunter,63.64%(11),41.67%(24),53.33%(15),48.94%(47),50.0%(18),60.0%(10),30.0%(30),100.0%(6),40.0%(10),25.0%(12),46.45%(183)
6,Hunter,66.67%(9),40.0%(5),100.0%(1),53.57%(28),40.0%(10),50.0%(8),61.11%(18),50.0%(2),100.0%(2),33.33%(12),52.63%(95)
7,Priest,28.57%(7),43.48%(23),55.56%(9),59.52%(42),70.0%(30),38.89%(18),50.0%(16),83.33%(6),42.86%(14),67.92%(53),57.34%(218)
8,Shaman,-,60.0%(5),0.0%(1),44.44%(18),0.0%(6),50.0%(2),16.67%(6),-,33.33%(3),50.0%(8),36.73%(49)
9,Warlock,40.0%(5),57.14%(7),100.0%(1),40.0%(20),60.0%(10),0.0%(2),57.14%(14),66.67%(3),50.0%(2),12.5%(8),45.83%(72)


,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(14),57.14%(14),36.36%(11),44.74%(38),40.0%(10),62.5%(8),54.55%(11),50.0%(2),66.67%(3),40.91%(22),47.37%(133)
2,Rogue,42.86%(14),50.0%(22),57.14%(14),39.53%(43),56.25%(16),60.0%(5),61.9%(21),75.0%(4),33.33%(9),28.57%(21),46.75%(169)
3,Mage,63.64%(11),42.86%(14),50.0%(10),68.0%(50),27.27%(11),0.0%(2),50.0%(22),33.33%(3),50.0%(8),26.09%(23),50.0%(154)
4,Druid,55.26%(38),60.47%(43),32.0%(50),50.0%(68),69.44%(36),73.33%(15),41.82%(55),54.55%(11),47.37%(19),55.56%(45),51.58%(380)
5,Demonhunter,60.0%(10),43.75%(16),72.73%(11),30.56%(36),50.0%(20),50.0%(8),48.48%(33),0.0%(1),50.0%(10),21.43%(14),44.03%(159)
6,Hunter,37.5%(8),40.0%(5),100.0%(2),26.67%(15),50.0%(8),-,40.0%(10),100.0%(1),60.0%(5),54.55%(11),44.62%(65)
7,Priest,45.45%(11),38.1%(21),50.0%(22),58.18%(55),51.52%(33),60.0%(10),50.0%(32),66.67%(6),37.5%(16),77.78%(45),55.78%(251)
8,Shaman,50.0%(2),25.0%(4),66.67%(3),45.45%(11),100.0%(1),0.0%(1),33.33%(6),50.0%(2),75.0%(4),14.29%(7),41.46%(41)
9,Warlock,33.33%(3),66.67%(9),50.0%(8),52.63%(19),50.0%(10),40.0%(5),62.5%(16),25.0%(4),50.0%(4),37.5%(8),51.16%(86)


...

,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(10),36.0%(25),50.0%(14),43.08%(65),52.94%(17),63.16%(19),41.03%(39),53.85%(13),13.33%(15),49.09%(55),44.85%(272)
2,Rogue,64.0%(25),50.0%(82),45.65%(46),46.56%(131),59.46%(37),46.43%(28),51.65%(91),62.5%(16),57.78%(45),44.44%(63),50.53%(564)
3,Mage,50.0%(14),54.35%(46),50.0%(24),45.78%(83),52.94%(17),41.18%(17),57.14%(56),58.33%(12),52.63%(19),40.62%(32),50.0%(320)
4,Druid,56.92%(65),53.44%(131),54.22%(83),50.0%(202),49.3%(71),56.9%(58),44.63%(121),70.0%(20),43.42%(76),48.06%(129),50.63%(956)
5,Demonhunter,47.06%(17),40.54%(37),47.06%(17),50.7%(71),50.0%(18),71.43%(14),37.21%(43),62.5%(8),64.29%(28),58.06%(31),50.35%(284)
6,Hunter,36.84%(19),53.57%(28),58.82%(17),43.1%(58),28.57%(14),50.0%(22),57.5%(40),75.0%(4),62.96%(27),42.11%(19),49.6%(248)
7,Priest,58.97%(39),48.35%(91),42.86%(56),55.37%(121),62.79%(43),42.5%(40),50.0%(52),55.0%(20),30.91%(55),46.97%(66),49.23%(583)
8,Shaman,46.15%(13),37.5%(16),41.67%(12),30.0%(20),37.5%(8),25.0%(4),45.0%(20),50.0%(4),60.0%(5),46.15%(13),40.87%(115)
9,Warlock,86.67%(15),42.22%(45),47.37%(19),56.58%(76),35.71%(28),37.04%(27),69.09%(55),40.0%(5),50.0%(26),47.83%(23),52.66%(319)


,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(2),25.0%(8),30.0%(10),62.16%(37),33.33%(12),50.0%(8),38.46%(13),100.0%(2),38.46%(13),51.85%(27),47.73%(132)
2,Rogue,75.0%(8),50.0%(62),56.0%(25),48.81%(84),52.17%(46),53.57%(28),59.09%(44),50.0%(2),47.62%(21),56.25%(32),52.84%(352)
3,Mage,70.0%(10),44.0%(25),50.0%(12),55.56%(36),58.97%(39),60.0%(10),61.76%(34),100.0%(1),53.85%(13),30.0%(30),52.86%(210)
4,Druid,37.84%(37),51.19%(84),44.44%(36),50.0%(138),51.47%(68),62.71%(59),41.56%(77),50.0%(12),52.63%(38),46.07%(89),49.06%(638)
5,Demonhunter,66.67%(12),47.83%(46),41.03%(39),48.53%(68),50.0%(34),46.15%(26),54.84%(31),100.0%(4),48.48%(33),50.0%(24),49.53%(317)
6,Hunter,50.0%(8),46.43%(28),40.0%(10),37.29%(59),53.85%(26),50.0%(20),47.06%(34),100.0%(2),51.85%(27),40.0%(15),45.85%(229)
7,Priest,61.54%(13),40.91%(44),38.24%(34),58.44%(77),45.16%(31),52.94%(34),50.0%(22),66.67%(6),43.24%(37),42.11%(38),48.51%(336)
8,Shaman,0.0%(2),50.0%(2),0.0%(1),50.0%(12),0.0%(4),0.0%(2),33.33%(6),-,0.0%(4),60.0%(5),31.58%(38)
9,Warlock,61.54%(13),52.38%(21),46.15%(13),47.37%(38),51.52%(33),48.15%(27),56.76%(37),100.0%(4),50.0%(16),23.53%(17),50.23%(219)


,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(6),28.57%(28),35.71%(14),46.51%(43),25.0%(20),61.54%(13),44.44%(9),-,0.0%(9),45.83%(24),38.55%(166)
2,Rogue,71.43%(28),50.0%(38),40.54%(37),54.72%(106),44.19%(43),66.67%(24),47.17%(53),100.0%(3),59.09%(22),45.24%(42),52.27%(396)
3,Mage,64.29%(14),59.46%(37),50.0%(16),50.98%(51),37.04%(27),46.15%(13),50.0%(20),50.0%(2),50.0%(16),61.54%(39),52.77%(235)
4,Druid,53.49%(43),45.28%(106),49.02%(51),50.0%(140),42.67%(75),58.82%(51),40.0%(60),100.0%(3),48.94%(47),55.07%(69),48.99%(645)
5,Demonhunter,75.0%(20),55.81%(43),62.96%(27),57.33%(75),50.0%(38),41.67%(24),50.0%(42),66.67%(3),52.0%(25),48.28%(29),54.6%(326)
6,Hunter,38.46%(13),33.33%(24),53.85%(13),41.18%(51),58.33%(24),50.0%(14),52.38%(21),66.67%(3),33.33%(15),50.0%(16),45.36%(194)
7,Priest,55.56%(9),52.83%(53),50.0%(20),60.0%(60),50.0%(42),47.62%(21),50.0%(22),66.67%(3),24.0%(25),65.52%(29),52.11%(284)
8,Shaman,-,0.0%(3),50.0%(2),0.0%(3),33.33%(3),33.33%(3),33.33%(3),50.0%(2),-,50.0%(6),32.0%(25)
9,Warlock,100.0%(9),40.91%(22),50.0%(16),51.06%(47),48.0%(25),66.67%(15),76.0%(25),-,50.0%(16),35.29%(17),54.69%(192)


,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(10),55.56%(9),20.0%(5),60.0%(10),46.15%(13),75.0%(8),50.0%(8),50.0%(2),20.0%(5),53.33%(15),50.59%(85)
2,Rogue,44.44%(9),50.0%(54),33.33%(15),53.33%(45),35.19%(54),59.26%(27),65.52%(29),100.0%(3),68.42%(19),50.0%(36),50.86%(291)
3,Mage,80.0%(5),66.67%(15),50.0%(2),33.33%(9),40.0%(15),28.57%(7),36.36%(11),100.0%(2),55.56%(9),52.94%(17),50.0%(92)
4,Druid,40.0%(10),46.67%(45),66.67%(9),50.0%(32),58.97%(39),42.11%(19),30.43%(23),-,46.67%(15),16.67%(18),45.24%(210)
5,Demonhunter,53.85%(13),64.81%(54),60.0%(15),41.03%(39),50.0%(46),58.06%(31),44.9%(49),100.0%(1),56.0%(25),37.5%(24),51.85%(297)
6,Hunter,25.0%(8),40.74%(27),71.43%(7),57.89%(19),41.94%(31),50.0%(22),48.28%(29),50.0%(4),40.0%(25),50.0%(14),46.24%(186)
7,Priest,50.0%(8),34.48%(29),63.64%(11),69.57%(23),55.1%(49),51.72%(29),50.0%(18),33.33%(6),26.92%(26),52.0%(25),49.11%(224)
8,Shaman,50.0%(2),0.0%(3),0.0%(2),-,0.0%(1),50.0%(4),66.67%(6),-,0.0%(2),50.0%(2),36.36%(22)
9,Warlock,80.0%(5),31.58%(19),44.44%(9),53.33%(15),44.0%(25),60.0%(25),73.08%(26),100.0%(2),50.0%(22),61.11%(18),54.82%(166)


,0,1,2,3,4,5,6,7,8,9,10,11
0,,Paladin,Rogue,Mage,Druid,Demonhunter,Hunter,Priest,Shaman,Warlock,Warrior,Total
1,Paladin,50.0%(8),34.78%(23),42.86%(7),68.75%(16),52.63%(19),61.54%(13),58.33%(12),100.0%(2),65.22%(23),45.45%(22),53.79%(145)
2,Rogue,65.22%(23),50.0%(148),53.49%(43),44.79%(96),37.63%(93),52.5%(40),62.96%(81),58.82%(17),66.67%(42),37.04%(54),50.24%(637)
3,Mage,57.14%(7),46.51%(43),50.0%(30),46.43%(28),34.62%(26),33.33%(9),56.0%(25),66.67%(3),58.82%(17),50.0%(26),48.13%(214)
4,Druid,31.25%(16),55.21%(96),53.57%(28),50.0%(78),52.63%(76),45.0%(40),43.1%(58),83.33%(6),33.33%(45),41.43%(70),47.56%(513)
5,Demonhunter,47.37%(19),62.37%(93),65.38%(26),47.37%(76),50.0%(56),68.75%(32),48.21%(56),57.14%(7),60.0%(45),50.0%(46),55.04%(456)
6,Hunter,38.46%(13),47.5%(40),66.67%(9),55.0%(40),31.25%(32),50.0%(30),54.29%(35),50.0%(2),42.86%(21),50.0%(26),47.98%(248)
7,Priest,41.67%(12),37.04%(81),44.0%(25),56.9%(58),51.79%(56),45.71%(35),50.0%(38),60.0%(5),30.0%(40),50.0%(56),45.81%(406)
8,Shaman,0.0%(2),41.18%(17),33.33%(3),16.67%(6),42.86%(7),50.0%(2),40.0%(5),50.0%(4),66.67%(3),55.56%(9),41.38%(58)
9,Warlock,34.78%(23),33.33%(42),41.18%(17),66.67%(45),40.0%(45),57.14%(21),70.0%(40),33.33%(3),50.0%(30),45.71%(35),49.5%(301)


# Metastats scrapping

## Accesing the content of the url and converting it to a soup

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows Phone 10.0; Android 6.0.1; Microsoft; RM-1152)'}
url = 'http://metastats.net/hearthstone/archetype/matchup/'
response = requests.get(url,headers = headers)
soup = BeautifulSoup(response.text)
print(soup)

<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/html">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Hearthstone Desck Archetype Matchup data and charts. Deck Matchup statistics for different archetype decks played on standard ladder for different ranks. You can choose All ranks, Rank 5 to Legend, Rank 10 to Rank 6" name="description"/>
<meta content="AdnanC" name="author"/>
<title>Hearthstone Deck Archetype Matchup  - Scholomance Academy - Hearthstone Meta Stats</title>
<link href="/favicon.png" rel="icon" type="image/png"/>
<!-- Latest compiled and minified CSS -->
<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" integrity="sha384-1q8mTJOASx8j1Au+a5WDVnPi2lkFfwwEAa8hDDdjZlpLegxhjVME1fgjWPGmkzs7" rel="stylesheet"/>
<!-- Optional theme -->
<!-- <link rel="stylesheet" href="https://maxcdn.boot

## Iterating through the contents and storing the values on a 2D matrix, convert it into a pandas dataframe and show results

In [6]:
table = soup.find('table')
data = []
# store the colums in data[0]
thread = table.find('thead') 
rows = table.find('tbody').find_all('tr',attrs={'class':'highlightrow'})
data.append(['']+[th.text for th in thread.find_all('th')])
decks_number = len(data[0])
for i in range(decks_number-1):
    row = rows[i]
    tds = row.find_all('td')
    results = []
    results.append(data[0][i+1])
    for j in range(1,decks_number):
        try:
            winrate = re.search('[0-9]+.?[0-9]+%',tds[j].findChildren()[0].get('title')).group()
        except AttributeError:
            winrate = '50%'
        results.append(winrate)
    data.append(results)
df = pd.DataFrame(data[1:], columns = data[0])
display(HTML(df.to_html()))


,,Aggro Demon Hunter,Control Demon Hunter,Tempo Demon Hunter,Token Demon Hunter,Dragon Druid,Gibberling Druid,Guardian Druid,Malygos Druid,Spell Druid,Dragon Hunter,Face Hunter,Highlander Hunter,Freeze Mage,Highlander Mage,No Minion Mage,Small Spell Mage,Tortollan Mage,Libram Paladin,Murloc Paladin,Galakrond Dragon Priest,Galakrond Priest,Highlander Priest,Resurrect Priest,Face Rogue,Galakrond Rogue,Galakrond Secret Rogue,Quest Rogue,Secret Rogue,Stealth Rogue,Weapon Rogue,Spell Damage Shaman,Totem Shaman,Galakrond Warlock,Hand Warlock,Zoo Warlock,Big Warrior,Bomb Warrior,Enrage Warrior,Galakrond Warrior,Pirate Warrior
0,Aggro Demon Hunter,50%,34%,47.22%,56.72%,52.81%,69.83%,46.56%,55.14%,64.7%,55.43%,60.39%,60%,84.15%,46.95%,58.58%,46.7%,59.07%,53.39%,65%,45.49%,49.62%,48.16%,52.52%,55.37%,53.79%,55.03%,56.38%,43.61%,52.36%,50.13%,60.23%,67.78%,54%,47.01%,61.29%,46.59%,59.33%,38.24%,64.41%,48.65%
1,Control Demon Hunter,65.99%,50%,69.94%,65.15%,35.13%,73.71%,35.59%,42%,50%,52.68%,58.94%,56.33%,76.35%,59.43%,52.97%,58.33%,69.55%,34.16%,60%,48.12%,50.38%,55.17%,42.42%,58.08%,50.63%,54.5%,63.92%,55.27%,50.31%,49.52%,54.4%,67.45%,60.69%,66.41%,59.39%,33.38%,38.96%,50.79%,39.67%,52.86%
2,Tempo Demon Hunter,52.77%,30.05%,50%,44.18%,45.94%,60%,41.97%,47.74%,60.31%,58.59%,53.7%,57.14%,75%,39.05%,43.39%,40.56%,61.48%,46.84%,61.72%,45.76%,52.5%,48.08%,50%,50%,50%,40.29%,54.79%,45.18%,47.7%,49.51%,58.22%,65.65%,46.07%,48.57%,53.96%,47.75%,60.31%,42.1%,58.06%,63.38%
3,Token Demon Hunter,43.27%,34.84%,55.81%,50%,33.78%,63.15%,29.51%,35.29%,44.82%,57.14%,45.06%,52.72%,63.93%,34.72%,47.57%,39.3%,50.61%,49.63%,63.33%,34.05%,37.7%,32.54%,44.2%,50.87%,36.53%,58.49%,47.16%,36.27%,43.9%,39.13%,56.6%,61.81%,37.14%,48.07%,39.79%,30.86%,27.08%,50%,50%,46.12%
4,Dragon Druid,47.18%,64.86%,54.05%,66.21%,50%,52.71%,47.55%,52.07%,67.24%,49.45%,44%,55%,55.11%,51.69%,60.4%,52.39%,42.72%,36.34%,36.66%,44.89%,45.83%,44.06%,37.8%,56.32%,65.08%,61.59%,53.22%,49%,41.3%,53.45%,59.74%,49.28%,50.18%,54.03%,40.65%,54.51%,46.21%,67.04%,46.4%,68.29%
5,Gibberling Druid,30.16%,26.28%,40%,36.84%,47.28%,50%,47.86%,57%,58.06%,48.58%,55.27%,59.09%,64.97%,41.91%,46.72%,45.93%,45.02%,42.83%,48.71%,43.52%,45.8%,46.64%,56.66%,56.1%,45.56%,45.09%,62.9%,41.89%,61.4%,53.17%,46.96%,54.65%,44.21%,45.94%,53.92%,46.97%,45.82%,35.06%,47.25%,50.68%
6,Guardian Druid,53.43%,64.4%,58.02%,70.48%,52.44%,52.13%,50%,59%,61.53%,54.77%,47.32%,58.73%,55.21%,52.14%,60.09%,57.22%,48.75%,45.89%,54.39%,46.71%,47.85%,43.92%,51.07%,57.58%,66.83%,64.52%,58.54%,49.95%,50.09%,57%,58.18%,51.83%,52.68%,56.55%,47.57%,49.26%,44.57%,69.56%,40.76%,65.2%
7,Malygos Druid,44.85%,57.99%,52.25%,64.7%,47.92%,42.99%,40.97%,50%,50%,47%,35.94%,49.25%,66.41%,59.13%,59.42%,49.86%,69.68%,35.72%,51.18%,51.05%,58.91%,56.36%,65.18%,51.44%,61.56%,59.47%,61.81%,42.89%,43.97%,48.89%,54.97%,39.76%,62.18%,72.32%,40%,48.24%,51.41%,54.86%,48.94%,49.6%
8,Spell Druid,35.29%,50%,39.68%,55.17%,32.75%,41.93%,38.46%,50%,50%,50%,40.64%,54.05%,63.49%,47.33%,55.55%,41.93%,49.01%,38.34%,59.61%,40.15%,56.61%,50%,49.01%,49.33%,56.36%,59.74%,48.14%,41.93%,45.79%,47.23%,45.2%,37.5%,50.63%,39.62%,37.14%,42.68%,47.08%,43.13%,47.05%,49.68%
9,Dragon Hunter,44.56%,47.31%,41.4%,42.85%,50.54%,51.41%,45.22%,52.99%,50%,50%,45.31%,50.35%,85.52%,60.3%,64.49%,53.11%,73.5%,38.91%,47.74%,55.28%,59.97%,55.68%,60.6%,56.48%,60.16%,46.05%,64.55%,51.03%,42.34%,50.04%,66.34%,52.96%,51.13%,60.86%,47.29%,55.74%,57.77%,47.27%,56.64%,51.11%


In [7]:
df.to_csv('output/Metastats.csv',index=False)